## 模块一：数据加载

你好！欢迎来到 R2Gen 项目的复现之旅。从今天开始，我们将一起探索如何利用深度学习技术，特别是 Transformer 模型，来生成医学影像报告。

作为整个项目的第一步，我们将从**数据加载**开始。数据是深度学习的“燃料”，如何高效、正确地把数据喂给模型，直接决定模型最终的性能。在 R2Gen 项目中，数据加载部分主要由 `datasets.py` 和 `dataloaders.py` 两个核心文件来完成。

### 为什么数据加载很重要？

1.  **数据预处理**：原始数据（比如图片、文本）通常不能直接被模型使用，需要进行一系列预处理操作，例如：
    *   **图像数据**：调整尺寸、归一化（将像素值缩放到特定范围）���数据增强（随机旋转、翻转等，增加数据多样性）。
    *   **文本数据**：分词（将句子切分成单词）、构建词典（将单词映射为数字 ID）、序列化（将文本转换为数字序列）。
2.  **批量处理（Batching）**：为了提高计算效率和模型训练的稳定性，我们通常不会一次只给模型一张图片或一个句子，而是将数据分成一个个的“批次”（Batch）。
3.  **并行化**：为了加快数据读取速度，我们会使用多个工作进程（Workers）来并行地加载数据。

### `datasets.py` 和 `dataloaders.py` 的分工

*   `datasets.py`：主要负责**定义数据集（Dataset）**。它告诉程序：
    *   数据存储在哪里？（`image_dir`, `ann_path`）
    *   数据一共有多少条？（`__len__` 方法）
    *   如何获取某一条具体的数据？（`__getitem__` 方法）
    *   在这个过程中，它会完成对单条数据的预处理，比如读取图片并进行变换、读取报告文本并转��为数字 ID 序列。

*   `dataloaders.py`：主要负责**创建数据加载器（DataLoader）**。它在 `Dataset` 的基础上，进一步实现了：
    *   将数据打包成批次（Batch）。
    *   通过多进程并行加载数据。
    *   对一个批次内的数据进行整理（Collate），比如将长度不同的文本序列填充到相同长度，方便模型处理。

接下来，让我们深入代码，看看 R2Gen 是如何实现这两个模块的。

### 1. `modules/datasets.py`

这个文件定义了两个核心的 `Dataset` 类：`IuxrayMultiImageDataset` 和 `MimiccxrSingleImageDataset`，它们都继承自一个通用的 `BaseDataset` 类。这对应了项目中使用的两个不同的数据集：IU X-Ray 和 MIMIC-CXR。

我们先来看代码，然后再逐一解释。

In [ ]:
import os
import json
import torch
from PIL import Image
from torch.utils.data import Dataset

class BaseDataset(Dataset):
    """
    基础的数据集类，定义了所有数据集共有的操作。
    """
    def __init__(self, args, tokenizer, split, transform=None):
        """
        初始化函数
        :param args: 包含所有配置参数的对象
        :param tokenizer: 用于处理文本的分词器
        :param split: 数据集划分，'train', 'val', 或 'test'
        :param transform: 应用于图像的预处理变换
        """
        self.image_dir = args.image_dir  # 图像文件夹路径
        self.ann_path = args.ann_path  # 标注文件路径 (annotation.json)
        self.max_seq_length = args.max_seq_length  # 报告文本的最大长度
        self.split = split  # 数据集划分
        self.tokenizer = tokenizer  # 分词器
        self.transform = transform  # 图像变换
        self.ann = json.loads(open(self.ann_path, 'r').read())  # 加载并解析标注文件

        # 根据 'train', 'val', 'test' 获取对应的样本
        self.examples = self.ann[self.split]
        # 遍历所有样本，进行预处理
        for i in range(len(self.examples)):
            # 使用分词器将报告文本转换为数字ID序列，并截断到最大长度
            self.examples[i]['ids'] = self.tokenizer(self.examples[i]['report'])[:self.max_seq_length]
            # 创建一个与ID序列等长的掩码（mask），用于在后续模型中告诉模型哪些是有效部分
            self.examples[i]['mask'] = [1] * len(self.examples[i]['ids'])

    def __len__(self):
        """
        返回数据集中样本的总数
        """
        return len(self.examples)


class IuxrayMultiImageDataset(BaseDataset):
    """
    针对 IU X-Ray 数据集的特定实现。这个数据集的特点是每个报告对应两张图片（正面和侧面）。
    """
    def __getitem__(self, idx):
        """
        获取并返回索引为 idx 的单个样本
        :param idx: 样本索引
        :return: 一个元组，包含 (image_id, image, report_ids, report_masks, seq_length)
        """
        example = self.examples[idx]
        image_id = example['id']  # 样本ID
        image_path = example['image_path']  # 图像路径列表，包含两张图
        
        # 分别读取两张图片，并转换为RGB格式
        image_1 = Image.open(os.path.join(self.image_dir, image_path[0])).convert('RGB')
        image_2 = Image.open(os.path.join(self.image_dir, image_path[1])).convert('RGB')
        
        # 如果定义了图像变换，则应用变换
        if self.transform is not None:
            image_1 = self.transform(image_1)
            image_2 = self.transform(image_2)
        
        # 将两张图片堆叠（stack）成一个张量（Tensor）
        image = torch.stack((image_1, image_2), 0)
        
        # 获取预处理好的报告ID和掩码
        report_ids = example['ids']
        report_masks = example['mask']
        seq_length = len(report_ids)  # 报告的实际长度
        
        sample = (image_id, image, report_ids, report_masks, seq_length)
        return sample


class MimiccxrSingleImageDataset(BaseDataset):
    """
    针对 MIMIC-CXR 数据集的特定实现。这个数据集每个报告只对应一张图片。
    """
    def __getitem__(self, idx):
        """
        获取并返回索引为 idx 的单个样本
        :param idx: 样本索引
        :return: 一个元组，包含 (image_id, image, report_ids, report_masks, seq_length)
        """
        example = self.examples[idx]
        image_id = example['id']
        image_path = example['image_path'] # 图像路径列表，只包含一张图
        
        # 读取单张图片
        image = Image.open(os.path.join(self.image_dir, image_path[0])).convert('RGB')
        
        # 应用图像变换
        if self.transform is not None:
            image = self.transform(image)
            
        # 获取报告ID和掩码
        report_ids = example['ids']
        report_masks = example['mask']
        seq_length = len(report_ids)
        
        sample = (image_id, image, report_ids, report_masks, seq_length)
        return sample

#### 代码讲解：

1.  **`BaseDataset`**: 这是两个特定数据集类的父类。它在 `__init__` 方法中完成了几件重要的事情：
    *   加载 `annotation.json` 文件，这个 JSON 文件里包含了所有样本的信息，比如图片路径、报告文本等。
    *   根据 `split` 参数（'train', 'val', 'test'）来选择要使用的数据子集。
    *   **核心预处理**：遍历所有样本，调用 `tokenizer` 将文本报告转换成一串数字（`ids`），并创建一个等长的 `mask`。这个 `mask` 都是1，用来告诉模型序列的哪些部分是有效的，哪些是后来为了对齐长度而填充的（padding）。

2.  **`IuxrayMultiImageDataset`**: 继承自 `BaseDataset`。它的特别之处在于 `__getitem__` 方法：
    *   它会根据 `image_path` 读取**两张**图片。
    *   对两张图片都进行 `transform`（图像变换）。
    *   使用 `torch.stack` 将两张处理后的图片合并成一个更高维度的张量。例如，如果每张图片是 `[3, 224, 224]`（3个颜色通道，224x224像素），堆叠后就变成了 `[2, 3, 224, 224]`，其中 `2` 代表两张不同的视图（正面/侧面）。

3.  **`MimiccxrSingleImageDataset`**: 同样继承自 `BaseDataset`，但它的 `__getitem__` 简单一些，只处理**一张**图片。

**知识点回顾：**
*   **`torch.utils.data.Dataset`**: 这是 PyTorch 中用来表示数据集的抽象类。任何自定义的数据集都需要继承它，并实现 `__len__` 和 `__getitem__` 这两个魔法方法。
*   **`__len__(self)`**: 应该返回数据集中样本的数量。
*   **`__getitem__(self, idx)`**: 应该返回数据集中索引为 `idx` 的一个样本。PyTorch 的 DataLoader 会调用这个方法来获取数据。
*   **`PIL.Image`**: 是一个强大的 Python 图像处理库，常用于读取和操作图片。
*   **`torch.stack`**: 沿着一个新的维度拼接张量序列。

### 2. `modules/dataloaders.py`

有了 `Dataset`，我们现在需要一个 `DataLoader` 来把它包装起来，实现批量加载和数据整理。`R2DataLoader` 就是为此而生。

In [ ]:
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
# 假设 datasets.py 中的类已经定义好
# from .datasets import IuxrayMultiImageDataset, MimiccxrSingleImageDataset

class R2DataLoader(DataLoader):
    """
    自定义的数据加载器，继承自 PyTorch 的 DataLoader。
    """
    def __init__(self, args, tokenizer, split, shuffle):
        """
        初始化函数
        :param args: 配置参数
        :param tokenizer: 分词器
        :param split: 数据集划分 ('train', 'val', 'test')
        :param shuffle: 是否在每个 epoch 开始时打乱数据顺序
        """
        self.args = args
        self.dataset_name = args.dataset_name
        self.batch_size = args.batch_size  # 批次大小
        self.shuffle = shuffle  # 是否打乱
        self.num_workers = args.num_workers  # 并行加载的进程数
        self.tokenizer = tokenizer
        self.split = split

        # 定义图像变换。训练集使用更复杂的变换（随机裁剪、翻转）来实现数据增强，
        # 而验证/测试集只进行必要的尺寸调整和归一化。
        if split == 'train':
            self.transform = transforms.Compose([
                transforms.Resize(256),  # 调整到256x256
                transforms.RandomCrop(224),  # 随机裁剪到224x224
                transforms.RandomHorizontalFlip(),  # 随机水平翻转
                transforms.ToTensor(),  # 转换为Tensor
                transforms.Normalize((0.485, 0.456, 0.406),  # 归一化
                                     (0.229, 0.224, 0.225))])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)), # 直接调整到224x224
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))])

        # 根据数据集名称，实例化对应的 Dataset 类
        if self.dataset_name == 'iu_xray':
            self.dataset = IuxrayMultiImageDataset(self.args, self.tokenizer, self.split, transform=self.transform)
        else:
            self.dataset = MimiccxrSingleImageDataset(self.args, self.tokenizer, self.split, transform=self.transform)

        # 设置 DataLoader 的初始化参数
        self.init_kwargs = {
            'dataset': self.dataset,
            'batch_size': self.batch_size,
            'shuffle': self.shuffle,
            'collate_fn': self.collate_fn,  # 指定自定义的整理函数
            'num_workers': self.num_workers
        }
        # 调用父类（DataLoader）的初始化方法
        super().__init__(**self.init_kwargs)

    @staticmethod
    def collate_fn(data):
        """
        静态方法，用于整理一个批次的数据。
        DataLoader 会从 Dataset 中取出 batch_size 个样本，组成一个列表 (data)，然后传给这个函数。
        :param data: 一个列表，列表的每个元素是 Dataset.__getitem__ 返回的元组。
        :return: 整理好的一个批次的数据
        """
        # 将 data 中的元素按类别解压
        images_id, images, reports_ids, reports_masks, seq_lengths = zip(*data)
        
        # 将图像列表堆叠成一个批次的张量
        images = torch.stack(images, 0)
        
        # 找到这个批次中报告的最大长度
        max_seq_length = max(seq_lengths)

        # 创建两个全零的 Numpy 数组，用于存放填充（padding）后的报告ID和掩码
        # 形状为 (批次大小, 最大长度)
        targets = np.zeros((len(reports_ids), max_seq_length), dtype=int)
        targets_masks = np.zeros((len(reports_ids), max_seq_length), dtype=int)

        # 遍历批次中的每个报告，将其复制到 targets 和 targets_masks 中
        for i, report_ids in enumerate(reports_ids):
            targets[i, :len(report_ids)] = report_ids

        for i, report_masks in enumerate(reports_masks):
            targets_masks[i, :len(report_masks)] = report_masks

        # 将 Numpy 数组转换为 PyTorch 的 Tensor 并返回
        return images_id, images, torch.LongTensor(targets), torch.FloatTensor(targets_masks)

#### 代码讲解：

1.  **`__init__`**: 
    *   **`transforms.Compose`**: 这是 torchvision 库中的一个类，可以将多个图像变换操作串联起来。值得注意的是，训练集（`split == 'train'`）的变换包含了 `RandomCrop` 和 `RandomHorizontalFlip`，这是一种常见的**数据增强（Data Augmentation）**手段，可以增加数据的多样性，防止模型过拟合，提高模型的泛化能力。
    *   **`transforms.Normalize`**: 归一化。它使用给定的均值和标准差对图像进行归一化。公式是 `output = (input - mean) / std`。这组特定的均值和标准差 `(0.485, 0.456, 0.406)` 和 `(0.229, 0.224, 0.225)` 是在 ImageNet 数据集上计算得出的，由于很多预训练模型（如 ResNet）都是在 ImageNet 上训练的，使用相同的归一化参数可以获得更好的性能。
    *   它会根据 `dataset_name` 来选择并实例化前面我们定义的 `IuxrayMultiImageDataset` 或 `MimiccxrSingleImageDataset`。
    *   最后，��调用了父类 `DataLoader` 的 `__init__` 方法，并传入了所有必要的参数，其中 `collate_fn` 是一个关键参数。

2.  **`collate_fn` (静态方法)**:
    *   这是 `DataLoader` 的精髓所在。当 `DataLoader` 从 `Dataset` 中取出一批（batch）数据后，这些数据是作为一个列表存在的，列表里每个元素都是 `__getitem__` 的返回值。但问题是，每个样本的报告长度 `seq_length` 可能不一样，导致 `reports_ids` 和 `reports_masks` 的长度也不同。这样的数据无法直接堆叠成一个规整的张量。
    *   `collate_fn` 的作用就是解决这个问题。它接收这个列表 `data`，然后：
        1.  找到这批数据中，报告文本的**最大长度** `max_seq_length`。
        2.  创建两个以 `max_seq_length` 为长度的、用0填充的数组 `targets` 和 `targets_masks`。
        3.  将每个样本的 `report_ids` 和 `report_masks` 复制到这两个数组中。对于那些长度小于 `max_seq_length` 的报告，它们后面的部分将保持为0，这个过程就叫做**��充（Padding）**。
        4.  最后，将整理好的、形状规整的数据转换成 PyTorch Tensor 返回。

**知识点回顾：**
*   **`torch.utils.data.DataLoader`**: PyTorch 核心的数据加载工具，实现了批量加载、数据打乱、多进程加载等功能。
*   **数据增强（Data Augmentation）**: 在训练过程中，对数据（尤其是图像）进行随机变换，以创造出更多样的训练样本，是提高模型性能和防止过拟合的常用技巧。
*   **归一化（Normalization）**: 将数据调整到相似的尺度，有助于加速模型训练和提高稳定性。
*   **填充（Padding）**: 在自然语言处理中，为了处理变长的序列数据，通常会将同一批次的所有序列填充到相同的长度。
*   **`collate_fn`**: `DataLoader` 中的一个可配置函数，专门用来定义如何将一个样本列表整理成一个批次的数据。当你需要处理变长序列等复杂数据时，自定义 `collate_fn` 是必不可少的。

---

到这里，我们就完成了 R2Gen 项目数据加载部分的核心代码学习。我们理解了 `Dataset` 如何定义和获取单条数据，以及 `DataLoader` 如何在此基础上实现批量化、并行化和数据整理。

**下一步预告：**
在下一个模块中，我们将学习 **`modules/tokenizers.py`**，深入了解文本是如何被清洗、切分并转换成数字序列的。这是连接自然语言和深度学习模型的桥梁。

如果你对今天的内容有任何疑问，随时可以提出来！